# San Francisco Crime Analysis (2018 until today)

San Francisco Crime Analysis is going to be performed as follows:

1. PySpark **environment setup**
2. Data source and **Spark data abstraction** (DataFrame) **set up**
3. Data set **metadata analysis**:
  1. Display **schema and size** of the DataFrame
  2. Changing the **names of**, and **deleting** Columns
  3. Changing **DayOfWeek** from string to integer
  4. Changing **time related** columns
  5. Get three **random samples** from the data set to better understand what the data is all about
  6. Identify **data entities**, **metrics** and **dimensions**
  7. **Column/field categorization**
4. Columns groups **basic profiling** to better understand our data set:
  1. **Timing related** columns basic profiling
  2. **Crime related** columns basic profiling
  3. **Location related** columns basic profiling
5. **Answer to business questions:** How can SFPD make the city of San Francisco safer?
  1. **Ratio of crime based on severity**
  2. **Severity of crimes and their location** by level of severity sorted by the most severe crimes (9)
      2. Severity of crimes and their location
      2. Severity of crimes and their location including most dangerous days of the week
  3. **Top 10 dangerous intersections with severe crime** 
      3. Top 10 dangerous intersections with any severe crime
      3. Top 10 dangerous intersections with any severe crime including days of the week

## 1. PySpark environment setup

In [17]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## 2. Data source and Spark data abstraction (DataFrame) setup

In [18]:
crimeDF = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("Police_Department_Incident_Reports__2018_to_Present.csv")

## 3. Data set metadata analysis

### A. Display schema and size of the DataFrame

In [19]:
from IPython.display import display, Markdown

crimeDF.printSchema()
display(Markdown("This DataFrame has **%d rows**." % crimeDF.count()))
display(Markdown("This DataFrame has **%d columns**." % len(crimeDF.columns)))

root
 |-- Incident Datetime: string (nullable = true)
 |-- Incident Date: string (nullable = true)
 |-- Incident Time: string (nullable = true)
 |-- Incident Year: integer (nullable = true)
 |-- Incident Day of Week: string (nullable = true)
 |-- Report Datetime: string (nullable = true)
 |-- Row ID: long (nullable = true)
 |-- Incident ID: integer (nullable = true)
 |-- Incident Number: integer (nullable = true)
 |-- CAD Number: integer (nullable = true)
 |-- Report Type Code: string (nullable = true)
 |-- Report Type Description: string (nullable = true)
 |-- Filed Online: boolean (nullable = true)
 |-- Incident Code: integer (nullable = true)
 |-- Incident Category: string (nullable = true)
 |-- Incident Subcategory: string (nullable = true)
 |-- Incident Description: string (nullable = true)
 |-- Resolution: string (nullable = true)
 |-- Intersection: string (nullable = true)
 |-- CNN: decimal(8,0) (nullable = true)
 |-- Police District: string (nullable = true)
 |-- Analysis Neigh

This DataFrame has **318743 rows**.

This DataFrame has **36 columns**.

### B. Changing the names of, and deleting Columns

Here I rename and remove columns as they appear in the above displayed schema. The few columns that I don't want to rename (Resolution, Intersection and Analysis Neighborhood) will not appear in this list:

In [20]:
crimeDF = crimeDF.withColumnRenamed("Incident Datetime", "DateAndTime")\
                 .withColumnRenamed("Incident Date", "Date")\
                 .withColumnRenamed("Incident Time", "Time")\
                 .withColumnRenamed("Incident Year", "Year")\
                 .withColumnRenamed("Incident Day of Week", "DayOfWeek")\
                 .withColumnRenamed("Report Datetime", "ReportDateAndTime")\
                 .withColumnRenamed("Row ID", "RowID")\
                 .withColumnRenamed("Incident ID", "IncidentID")\
                 .withColumnRenamed("Incident Number", "IncidentNumber")\
                 .withColumnRenamed("CAD Number", "CADNumber")\
                 .withColumnRenamed("Report Type Code", "ReportTypeCode")\
                 .withColumnRenamed("Report Type Description", "ReportTypeDescription")\
                 .withColumnRenamed("Filed Online", "Online")\
                 .withColumnRenamed("Incident Code", "IncidentCode")\
                 .withColumnRenamed("Incident Category", "Category")\
                 .withColumnRenamed("Incident Subcategory", "Subcategory")\
                 .withColumnRenamed("Incident Description", "Description")\
                 .withColumnRenamed("Incident Code", "Code")\
                 .withColumnRenamed("CNN", "IntersectionID")\
                 .withColumnRenamed("Police District", "PoliceDistrict")\
                 .withColumnRenamed("Analysis Neighborhood", "Neighborhood")\
                 .withColumnRenamed("Supervisor District", "SupervisorDistrict")\
                 .withColumnRenamed("SF Find Neighborhoods", "NeighborhoodID")\
                 .withColumnRenamed("Current Police Districts", "PoliceDistrictID")\
                 .withColumnRenamed("HSOC Zones as of 2018-06-05", "HSOCZones")\
                 .withColumnRenamed("OWED Public Spaces", "OWEDPublicSpaces")\
                 .withColumnRenamed("Central Market/Tenderloin Boundary Polygon - Updated", "CentralMarket/TenderloinBoundaryPolygon")\
                 .withColumnRenamed("Parks Alliance CPSI (27+TL sites)", "ParksAllianceCPSI")\
                 .withColumnRenamed("ESNCAG - Boundary File", "ESNCAG-BoundaryFile")\
                 .withColumnRenamed("Current Supervisor Districts", "CurrentSupervisorDistricts")\
                 .withColumnRenamed("Areas of Vulnerability, 2016", "AreasOfVulnerability")\
                 .drop("Latitude", "Longitude", "point", "Analysis Neighborhoods")

### C. Changing DayOfWeek from string to integer values

In [21]:
from pyspark.sql.functions import col, when

crimeDF = crimeDF.withColumn("DayOfWeek", 
                               when(col("DayOfWeek") == "Monday", 1)\
                              .when(col("DayOfWeek") == "Tuesday", 2)\
                              .when(col("DayOfWeek") == "Wednesday", 3)\
                              .when(col("DayOfWeek") == "Thursday", 4)\
                              .when(col("DayOfWeek") == "Friday", 5)\
                              .when(col("DayOfWeek") == "Saturday", 6)\
                              .when(col("DayOfWeek") == "Sunday", 7))

### D. Changing time related columns

In [22]:
#we see that the type of Date is a string, so to be able to add the column "DayOfWeek", we have to transform it to a Date type and apply the function
from datetime import datetime
from pyspark.sql.functions import udf, date_format, year, month, dayofmonth, hour
from pyspark.sql.types import DateType, IntegerType

func =  udf (lambda x: datetime.strptime(x, "%Y/%m/%d"), DateType())

crimeDF = crimeDF.withColumn("DateNew", func(col("Date")))

crimeDF = crimeDF.select("DateAndTime",
                         "Date",
                         month("DateNew").alias("Month"),
                         dayofmonth("DateNew").alias("DayOfMonth"),
                         "Time",
                         hour("Time").alias("Hour"),
                         "Year",
                         "DayOfWeek",
                         "ReportDateAndTime",
                         "RowID",
                         "IncidentID",
                         "IncidentNumber",
                         "CADNumber",
                         "ReportTypeCode",
                         "ReportTypeDescription",
                         "Online",
                         "IncidentCode",
                         "Category",
                         "Subcategory",
                         "Description",
                         "Resolution",
                         "Intersection",
                         "IntersectionID",
                         "PoliceDistrict",
                         "Neighborhood",
                         "SupervisorDistrict",
                         "NeighborhoodID",
                         "PoliceDistrictID",
                         "CurrentSupervisorDistricts",
                         "HSOCZones",
                         "OWEDPublicSpaces",
                         "CentralMarket/TenderloinBoundaryPolygon",
                         "ParksAllianceCPSI",
                         "ESNCAG-BoundaryFile",
                         "AreasOfVulnerability")

### E. Get three random samples from the data set

In [23]:
crimeDF.cache() # optimization to make the processing faster
crimeDF.sample(False, 0.1).take(3)

[Row(DateAndTime='2020/02/02 07:36:00 PM', Date='2020/02/02', Month=2, DayOfMonth=2, Time='19:36', Hour=19, Year=2020, DayOfWeek=7, ReportDateAndTime='2020/02/02 07:40:00 PM', RowID=89854404013, IncidentID=898544, IncidentNumber=200083012, CADNumber=200332914, ReportTypeCode='II', ReportTypeDescription='Initial', Online=None, IncidentCode=4013, Category='Assault', Subcategory='Aggravated Assault', Description='Assault, Aggravated, W/ Other Weapon', Resolution='Cite or Arrest Adult', Intersection='HILLVIEW CT \\ CASHMERE ST', IntersectionID=Decimal('33634000'), PoliceDistrict='Bayview', Neighborhood='Bayview Hunters Point', SupervisorDistrict=10, NeighborhoodID=86, PoliceDistrictID=2, CurrentSupervisorDistricts=9, HSOCZones=None, OWEDPublicSpaces=None, CentralMarket/TenderloinBoundaryPolygon=None, ParksAllianceCPSI=None, ESNCAG-BoundaryFile=None, AreasOfVulnerability=2),
 Row(DateAndTime='2020/02/03 07:00:00 PM', Date='2020/02/03', Month=2, DayOfMonth=3, Time='19:00', Hour=19, Year=2020

### F. Data entities, metrics and dimensions

I've identified the following elements:

* **Entities:** IncidentID, PoliceDistrict, Neighborhood
* **Metrics:** DateAndTime, Date, Month, DayOfMonth, Time, Hour, Year, DayOfWeek, ReportDateAndTime
* **Dimensions:** RowID, IncidentNumber, CADNumber, ReportTypeCode, ReportTypeDescription, Online, IncidentCode, Category, Subcategory, Description, Resolution, Intersection, IntersectionID, PoliceDistrict, Neighborhood, SupervisorDistrict, NeighborhoodID, PoliceDistrictID, CurrentSupervisorDistrict, HSOCZones, OWEDPublicSpaces, CentralMarket/TenderloinBoundaryPolygon, ParksAllianceCPSI, ESNCAG-BoundaryFile and AreasOfVulnerability

### G. Column categorization

The following could be a potential column categorization:

* **Timing related columns:** *DateAndTime, Date, Month, DayOfMonth, Time, Hour, Year, DayOfWeek* and *ReportDateAndTime*
* **Crime related columns:** *IncidentID, CADNumber, ReportTypeCode, ReportTypeDescription, Online, IncidentCode, Category, Subcategory, Description* and *Resolution*
* **Location related columns:** *Neighborhood, Intersection, IntersectionID, PoliceDistrict, Neighborhood, SupervisorDistrict, NeighborhoodID, PoliceDistrictID, CurrentSupervisorDistrict, HSOCZones, OWEDPublicSpaces, CentralMarket/TenderloinBoundaryPolygon, ParksAllianceCPSI, ESNCAG-BoundaryFile* and *AreasOfVulnerability*


## 4. Columns groups basic profiling to better understand our data set

### A. Timing related columns basic profiling

In [24]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit, date_format

print ("Summary of columns Year, Month, DayOfMonth, DayOfWeek and Hour:")
crimeDF.select("Year", "Month", "DayOfMonth", "DayOfWeek", "Hour").summary().show()

print("Checking for nulls on columns Year, Month, DayOfMonth, DayOfWeek and Hour:")
crimeDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Year", "Month", "DayOfMonth", "DayOfWeek", "Hour"]]).show()

print("Checking amount of distinct values in columns Year, Month, DayOfMonth, DayOfWeek and Hour:")
crimeDF.select([countDistinct(c).alias(c) for c in ["Year", "Month", "DayOfMonth", "DayOfWeek", "Hour"]]).show()

print ("Most and least frequent occurrences for Year, Month, DayOfMonth, DayOfWeek and Hour:")
yearDF = crimeDF.groupBy("Year").agg(count(lit(1)).alias("Total"))
monthDF = crimeDF.groupBy("Month").agg(count(lit(1)).alias("Total"))
dayOfMonthDF = crimeDF.groupBy("DayOfMonth").agg(count(lit(1)).alias("Total"))
dayOfWeekDF = crimeDF.groupBy("DayOfWeek").agg(count(lit(1)).alias("Total"))
hourDF = crimeDF.groupBy("Hour").agg(count(lit(1)).alias("Total"))

leastFreqYear       = yearDF.orderBy(col("Total").asc()).first()
mostFreqYear        = yearDF.orderBy(col("Total").desc()).first()
leastFreqMonth      = monthDF.orderBy(col("Total").asc()).first()
mostFreqMonth       = monthDF.orderBy(col("Total").desc()).first()
leastFreqDayOfMonth = dayOfMonthDF.orderBy(col("Total").asc()).first()
mostFreqDayOfMonth  = dayOfMonthDF.orderBy(col("Total").desc()).first()
leastFreqDayOfWeek  = dayOfWeekDF.orderBy(col("Total").asc()).first()
mostFreqDayOfWeek   = dayOfWeekDF.orderBy(col("Total").desc()).first()
leastFreqHour       = hourDF.orderBy(col("Total").asc()).first()
mostFreqHour        = hourDF.orderBy(col("Total").desc()).first()

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqYear", "mostFreqYear", "leastFreqMonth", "mostFreqMonth",\
       "%d (%s occurrences)" % (leastFreqYear["Year"], leastFreqYear["Total"]), \
       "%d (%s occurrences)" % (mostFreqYear["Year"], mostFreqYear["Total"]), \
       "%d (%s occurrences)" % (leastFreqMonth["Month"], leastFreqMonth["Total"]), \
       "%d (%s occurrences)" % (mostFreqMonth["Month"], mostFreqMonth["Total"]))))

display(Markdown("""
| %s | %s | %s | %s | %s | %s |
|----|----|----|----|----|----|
| %s | %s | %s | %s | %s | %s |
""" % ("leastFreqDayOfMonth", "mostFreqDayOfMonth", "leastFreqDayOfWeek", "mostFreqDayOfWeek", "leastFreqHour", "mostFreqHour",\
       "%d (%s occurrences)" % (leastFreqDayOfMonth["DayOfMonth"], leastFreqDayOfMonth["Total"]), \
       "%d (%s occurrences)" % (mostFreqDayOfMonth["DayOfMonth"], mostFreqDayOfMonth["Total"]), \
       "%d (%s occurrences)" % (leastFreqDayOfWeek["DayOfWeek"], leastFreqDayOfWeek["Total"]), \
       "%d (%s occurrences)" % (mostFreqDayOfWeek["DayOfWeek"], mostFreqDayOfWeek["Total"]), \
       "%d (%s occurrences)" % (leastFreqHour["Hour"], leastFreqHour["Total"]), \
       "%d (%s occurrences)" % (mostFreqHour["Hour"], mostFreqHour["Total"]))))

Summary of columns Year, Month, DayOfMonth, DayOfWeek and Hour:
+-------+------------------+------------------+------------------+-----------------+------------------+
|summary|              Year|             Month|        DayOfMonth|        DayOfWeek|              Hour|
+-------+------------------+------------------+------------------+-----------------+------------------+
|  count|            318743|            318743|            318743|           318743|            318743|
|   mean|2018.5706195900773| 6.283221906049701|15.578337406625401|3.977357934135024|13.257906840307081|
| stddev| 0.591704587798283|3.5289639744465875| 8.850452226959106|1.974765344051228| 6.364420847821066|
|    min|              2018|                 1|                 1|                1|                 0|
|    25%|              2018|                 3|                 8|                2|                 9|
|    50%|              2019|                 6|                15|                4|                14|



| leastFreqYear | mostFreqYear | leastFreqMonth | mostFreqMonth |
|----|----|----|----|
| 2020 (16750 occurrences) | 2018 (153612 occurrences) | 11 (24116 occurrences) | 1 (37088 occurrences) |



| leastFreqDayOfMonth | mostFreqDayOfMonth | leastFreqDayOfWeek | mostFreqDayOfWeek | leastFreqHour | mostFreqHour |
|----|----|----|----|----|----|
| 31 (6393 occurrences) | 1 (11859 occurrences) | 7 (42001 occurrences) | 5 (48753 occurrences) | 5 (3973 occurrences) | 12 (20051 occurrences) |


### B. Crime related columns basic profiling

In [25]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first

print ("Summary of columns IncidentID, CADNumber, ReportTypeCode, ReportTypeDescription, Online and IncidentCode:")
crimeDF.select("IncidentID", "CADNumber", "ReportTypeCode", "ReportTypeDescription", "Online", "IncidentCode").summary().show()

print("Checking for nulls on columns IncidentID, CADNumber, ReportTypeCode, ReportTypeDescription, Online and IncidentCode:")
crimeDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["IncidentID", "CADNumber", "ReportTypeCode", "ReportTypeDescription", "Online", "IncidentCode"]]).show()

print("Checking amount of distinct values in columns IncidentID, CADNumber, ReportTypeCode, ReportTypeDescription, Online and IncidentCode:")
crimeDF.select([countDistinct(c).alias(c) for c in ["IncidentID", "CADNumber", "ReportTypeCode", "ReportTypeDescription", "Online", "IncidentCode"]]).show()

print ("Most and least frequent occurrences for ReportTypeCode, ReportTypeDescription and IncidentCode columns:")
reportTypeCodeDF          = crimeDF.groupBy("ReportTypeCode").agg(count(lit(1)).alias("Total"))
reportTypeDescriptionDF   = crimeDF.groupBy("ReportTypeDescription").agg(count(lit(1)).alias("Total"))
incidentCodeDF            = crimeDF.groupBy("IncidentCode").agg(count(lit(1)).alias("Total"))


leastFreqReportTypeCode         = reportTypeCodeDF.orderBy(col("Total").asc()).first()
mostFreqReportTypeCode          = reportTypeCodeDF.orderBy(col("Total").desc()).first()
leastFreqReportTypeDescription  = reportTypeDescriptionDF.orderBy(col("Total").asc()).first()
mostFreqReportTypeDescription   = reportTypeDescriptionDF.orderBy(col("Total").desc()).first()
leastFreqIncidentCode           = incidentCodeDF.orderBy(col("Total").asc()).first()
mostFreqIncidentCode            = incidentCodeDF.orderBy(col("Total").desc()).first()


display(Markdown("""
| %s | %s | %s | %s | %s | %s |
|----|----|----|----|----|----|
| %s | %s | %s | %s | %s | %s |
""" % ("leastFreqReportTypeCode", "mostFreqReportTypeCode", "leastFreqReportTypeDescription",
       "mostFreqReportTypeDescription", "leastFreqIncidentCode", "mostFreqIncidentCode", \
       "%s (%d occurrences)" % (leastFreqReportTypeCode["ReportTypeCode"], leastFreqReportTypeCode["Total"]), \
       "%s (%d occurrences)" % (mostFreqReportTypeCode["ReportTypeCode"], mostFreqReportTypeCode["Total"]), \
       "%s (%d occurrences)" % (leastFreqReportTypeDescription["ReportTypeDescription"], leastFreqReportTypeDescription["Total"]), \
       "%s (%d occurrences)" % (mostFreqReportTypeDescription["ReportTypeDescription"], mostFreqReportTypeDescription["Total"]), \
       "%s (%d occurrences)" % (leastFreqIncidentCode["IncidentCode"], leastFreqIncidentCode["Total"]), \
       "%s (%d occurrences)" % (mostFreqIncidentCode["IncidentCode"], mostFreqIncidentCode["Total"]))))

print ("Summary of columns Category, Subcategory, Description and Resolution:")
crimeDF.select("Category", "Subcategory", "Description", "Resolution").summary().show()

print("Checking for nulls on columns Category, Subcategory, Description and Resolution:")
crimeDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Category", "Subcategory", "Description", "Resolution"]]).show()

print("Checking amount of distinct values in columns Category, Subcategory, Description and Resolution:")
crimeDF.select([countDistinct(c).alias(c) for c in ["Category", "Subcategory", "Description", "Resolution"]]).show()

print ("Most and least frequent occurrences for Category, Subcategory, Description and Resolution columns:")
categoryDF      = crimeDF.groupBy("Category").agg(count(lit(1)).alias("Total"))
subcategoryDF   = crimeDF.groupBy("Subcategory").agg(count(lit(1)).alias("Total"))
descriptionDF   = crimeDF.groupBy("Description").agg(count(lit(1)).alias("Total"))
resolutionDF    = crimeDF.groupBy("Resolution").agg(count(lit(1)).alias("Total"))


leastFreqCategory     = categoryDF.orderBy(col("Total").asc()).first()
mostFreqCategory      = categoryDF.orderBy(col("Total").desc()).first()
leastFreqSubcategory  = subcategoryDF.orderBy(col("Total").asc()).first()
mostFreqSubcategory   = subcategoryDF.orderBy(col("Total").desc()).first()
leastFreqDescription  = descriptionDF.orderBy(col("Total").asc()).first()
mostFreqDescription   = descriptionDF.orderBy(col("Total").desc()).first()
leastFreqResolution   = resolutionDF.orderBy(col("Total").asc()).first()
mostFreqResolution    = resolutionDF.orderBy(col("Total").desc()).first()


display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqCategory", "mostFreqCategory", "leastFreqSubcategory", "mostFreqSubcategory", \
       "%s (%d occurrences)" % (leastFreqCategory["Category"], leastFreqCategory["Total"]), \
       "%s (%d occurrences)" % (mostFreqCategory["Category"], mostFreqCategory["Total"]), \
       "%s (%d occurrences)" % (leastFreqSubcategory["Subcategory"], leastFreqSubcategory["Total"]), \
       "%s (%d occurrences)" % (mostFreqSubcategory["Subcategory"], mostFreqSubcategory["Total"]))))

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqDescription", "mostFreqDescription", "leastFreqResolution", "mostFreqDest", \
       "%s (%d occurrences)" % (leastFreqDescription["Description"], leastFreqDescription["Total"]), \
       "%s (%d occurrences)" % (mostFreqDescription["Description"], mostFreqDescription["Total"]), \
       "%s (%d occurrences)" % (leastFreqResolution["Resolution"], leastFreqResolution["Total"]), \
       "%s (%d occurrences)" % (mostFreqResolution["Resolution"], mostFreqResolution["Total"]))))

Summary of columns IncidentID, CADNumber, ReportTypeCode, ReportTypeDescription, Online and IncidentCode:
+-------+-----------------+--------------------+--------------+---------------------+------------------+
|summary|       IncidentID|           CADNumber|ReportTypeCode|ReportTypeDescription|      IncidentCode|
+-------+-----------------+--------------------+--------------+---------------------+------------------+
|  count|           318743|              244493|        318743|               318743|            318743|
|   mean|762313.4191527343| 1.877710446638922E8|          null|                 null| 25315.26968121653|
| stddev|81580.24453852163|1.3837205409779647E7|          null|                 null|25944.330989178667|
|    min|           618687|                   1|            II|     Coplogic Initial|              1000|
|    25%|           692045|           181970602|          null|                 null|              6244|
|    50%|           762409|           190213490|      


| leastFreqReportTypeCode | mostFreqReportTypeCode | leastFreqReportTypeDescription | mostFreqReportTypeDescription | leastFreqIncidentCode | mostFreqIncidentCode |
|----|----|----|----|----|----|
| VS (9941 occurrences) | II (260278 occurrences) | Coplogic Supplement (7795 occurrences) | Initial (198802 occurrences) | 9162 (1 occurrences) | 6244 (44601 occurrences) |


Summary of columns Category, Subcategory, Description and Resolution:
+-------+---------------+------------------+--------------------+--------------------+
|summary|       Category|       Subcategory|         Description|          Resolution|
+-------+---------------+------------------+--------------------+--------------------+
|  count|         318723|            318723|              318743|              318743|
|   mean|           null|              null|                null|                null|
| stddev|           null|              null|                null|                null|
|    min|          Arson|Aggravated Assault|"Fireworks, Posse...|Cite or Arrest Adult|
|    25%|           null|              null|                null|                null|
|    50%|           null|              null|                null|                null|
|    75%|           null|              null|                null|                null|
|    max|Weapons Offense|   Weapons Offense|Wiretaps, Unauth


| leastFreqCategory | mostFreqCategory | leastFreqSubcategory | mostFreqSubcategory |
|----|----|----|----|
| Human Trafficking (B), Involuntary Servitude (1 occurrences) | Larceny Theft (102993 occurrences) | Human Trafficking (A), Commercial Sex Acts (1 occurrences) | Larceny - From Vehicle (59744 occurrences) |



| leastFreqDescription | mostFreqDescription | leastFreqResolution | mostFreqDest |
|----|----|----|----|
| Amphetamine, Transportation (1 occurrences) | Theft, From Locked Vehicle, >$950 (44601 occurrences) | Exceptional Adult (872 occurrences) | Open or Active (243946 occurrences) |


### C. Location related columns basic profiling

In [26]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first

print ("Summary of columns Neighborhood, Intersection, IntersectionID and PoliceDistrict:")
crimeDF.select("Neighborhood", "Intersection", "IntersectionID", "PoliceDistrict").summary().show()

print("Checking for nulls on columns Neighborhood, Intersection, IntersectionID and PoliceDistrict:")
crimeDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["Neighborhood", "Intersection", "IntersectionID", "PoliceDistrict"]]).show()

print("Checking amount of distinct values in columns Neighborhood, Intersection, IntersectionID and PoliceDistrict:")
crimeDF.select([countDistinct(c).alias(c) for c in ["Neighborhood", "Intersection", "IntersectionID", "PoliceDistrict"]]).show()

print ("Summary of columns SupervisorDistrict, NeighborhoodID, PoliceDistrictID, CurrentSupervisorDistricts and HSOCZones:")
crimeDF.select("SupervisorDistrict", "NeighborhoodID", "PoliceDistrictID", "CurrentSupervisorDistricts", "HSOCZones").summary().show()

print("Checking for nulls on columns SupervisorDistrict, NeighborhoodID, PoliceDistrictID, CurrentSupervisorDistricts and HSOCZones:")
crimeDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["SupervisorDistrict", "NeighborhoodID", "PoliceDistrictID", "CurrentSupervisorDistricts", "HSOCZones"]]).show()

print("Checking amount of distinct values in columns SupervisorDistrict, NeighborhoodID, PoliceDistrictID, CurrentSupervisorDistricts and HSOCZones")
crimeDF.select([countDistinct(c).alias(c) for c in ["SupervisorDistrict", "NeighborhoodID", "PoliceDistrictID", "CurrentSupervisorDistricts", "HSOCZones"]]).show()

print ("Summary of columns OWEDPublicSpaces, CentralMarket/TenderloinBoundaryPolygon, ParksAllianceCPSI, ESNCAG-BoundaryFile and AreasOfVulnerability:")
crimeDF.select("OWEDPublicSpaces", "CentralMarket/TenderloinBoundaryPolygon", "ParksAllianceCPSI", "ESNCAG-BoundaryFile", "AreasOfVulnerability").summary().show()

print("Checking for nulls on columns OWEDPublicSpaces, CentralMarket/TenderloinBoundaryPolygon, ParksAllianceCPSI, ESNCAG-BoundaryFile and AreasOfVulnerability:")
crimeDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["OWEDPublicSpaces", "CentralMarket/TenderloinBoundaryPolygon", "ParksAllianceCPSI", "ESNCAG-BoundaryFile", "AreasOfVulnerability"]]).show()

print("Checking amount of distinct values in columns OWEDPublicSpaces, CentralMarket/TenderloinBoundaryPolygon, ParksAllianceCPSI, ESNCAG-BoundaryFile and AreasOfVulnerability:")
crimeDF.select([countDistinct(c).alias(c) for c in ["OWEDPublicSpaces", "CentralMarket/TenderloinBoundaryPolygon", "ParksAllianceCPSI", "ESNCAG-BoundaryFile", "AreasOfVulnerability"]]).show()

Summary of columns Neighborhood, Intersection, IntersectionID and PoliceDistrict:
+-------+--------------------+--------------------+------------------+--------------+
|summary|        Neighborhood|        Intersection|    IntersectionID|PoliceDistrict|
+-------+--------------------+--------------------+------------------+--------------+
|  count|              301645|              301645|            301645|        318743|
|   mean|                null|                null|     25358362.7575|          null|
| stddev|                null|                null|3088357.4889258808|          null|
|    min|Bayview Hunters P...|01ST ST \ BUSH ST...|          20013000|       Bayview|
|    25%|                null|                null|         2.4021E+7|          null|
|    50%|                null|                null|         2.4915E+7|          null|
|    75%|                null|                null|         2.6412E+7|          null|
|    max|                null|  ZOE ST \ BRYANT ST|       

## 5. Answer to business questions: How can SFPD make the city of San Francisco safer?

### A. Ratio of crime based on severity

In [27]:
from pyspark.sql.functions import count, round

# Severity is going to be categorized into 10 different levels based on prison sentences and severity of the crime
# from a document from the US (https://pap.georgia.gov/sites/pap.georgia.gov/files/CSL-s_Post_1-1-2006_considerations.pdf).
# If the subcategory is not found, I had to use my intuition of how severe of a crime it is:

#   "Level 0" - not necessarily a Crime
#   "Level 1" - least severe crimes
#   "Level 2"
#   "Level 3"
#   "Level 4"
#   "Level 5"
#   "Level 6"
#   "Level 7"
#   "Level 8"
#   "Level 9" - most severe crimes

# 1. Lets see all the distinct values of Subcategory. Subcategory is used instead of category due to more distinct values.
crimeDF.select("subcategory").distinct().show(75, False)

# 2. Lets enrich the crimeDF with severity based on our categorization
totalCrimes = crimeDF.count()
crimeDF = crimeDF\
   .where(col("subcategory")!="NA")\
   .withColumn("CrimeSeverity", when(col("subcategory") == "Burglary - Hot Prowl",8)\
                               .when(col("subcategory") == "LiquorLaw Violation",1)\
                               .when(col("subcategory") == "Burglary - Commercial",3)\
                               .when(col("subcategory") == "Vehicle Misplaced",1)\
                               .when(col("subcategory") == "Child Abuse",8)\
                               .when(col("subcategory") == "Stalking",8)\
                               .when(col("subcategory") == "Larceny Theft – From Building",2)\
                               .when(col("subcategory") == "Drunkenness",2)\
                               .when(col("subcategory") == "Forgery And Counterfeiting",3)\
                               .when(col("subcategory") == "Robbery - Other",8)\
                               .when(col("subcategory") == "Sex Offense",7)\
                               .when(col("subcategory") == "Intimidation",4)\
                               .when(col("subcategory") == "Fire Report",3)\
                               .when(col("subcategory") == "Simple Assault",5)\
                               .when(col("subcategory") == "Larceny Theft - Bicycle",3)\
                               .when(col("subcategory") == "Larceny – Auto Parts",2)\
                               .when(col("subcategory") == "Robbery - Carjacking",8)\
                               .when(col("subcategory") == "Recovered Vehicle",0)\
                               .when(col("subcategory") == "Drug Violation",2)\
                               .when(col("subcategory") == "Embezzlement",3)\
                               .when(col("subcategory") == "Vehicle Impounded",2)\
                               .when(col("subcategory") == "null",0)\
                               .when(col("subcategory") == "Theft From Vehicle",4)\
                               .when(col("subcategory") == "Missing Person",0)\
                               .when(col("subcategory") == "Rape",9)\
                               .when(col("subcategory") == "Human Trafficking, Commercial Sex Acts",7)\
                               .when(col("subcategory") == "Bribery",6)\
                               .when(col("subcategory") == "Lost Property",0)\
                               .when(col("subcategory") == "Arson",5)\
                               .when(col("subcategory") == "Fraud",5)\
                               .when(col("subcategory") == "Homicide",9)\
                               .when(col("subcategory") == "Loitering",5)\
                               .when(col("subcategory") == "Bad Checks",6)\
                               .when(col("subcategory") == "Larceny Theft - Pickpocket",4)\
                               .when(col("subcategory") == "Suspicious Package",0)\
                               .when(col("subcategory") == "Larceny Theft - Shoplifting",4)\
                               .when(col("subcategory") == "Robbery - Residential",6)\
                               .when(col("subcategory") == "Suicide",0)\
                               .when(col("subcategory") == "LarcenyTheft - Other",5)\
                               .when(col("subcategory") == "Sex Offense, Child",8)\
                               .when(col("subcategory") == "Other",1)\
                               .when(col("subcategory") == "Robbery - Street",5)\
                               .when(col("subcategory") == "Gambling",3)\
                               .when(col("subcategory") == "Manslaughter",8)\
                               .when(col("subcategory") == "Arrest",3)\
                               .when(col("subcategory") == "Human Trafficking(A), Commercial Sex Acts",8)\
                               .when(col("subcategory") == "Larceny - From Vehicle",4)\
                               .when(col("subcategory") == "Warrant",3)\
                               .when(col("subcategory") == "Traffic Violation Arrest",3)\
                               .when(col("subcategory") == "Courtesy Report",0)\
                               .when(col("subcategory") == "Trespass",3)\
                               .when(col("subcategory") == "Traffic Collision",3)\
                               .when(col("subcategory") == "Rape - Attempted",8)\
                               .when(col("subcategory") == "Family Offenses",4)\
                               .when(col("subcategory") == "Traffic Collision – Hit & Run",4)\
                               .when(col("subcategory") == "Weapons Offense",4)\
                               .when(col("subcategory") == "Other Offenses",1)\
                               .when(col("subcategory") == "Larceny Theft – Purse Snatch",1)\
                               .when(col("subcategory") == "Burglary - Other",1)\
                               .when(col("subcategory") == "Kidnapping",9)\
                               .when(col("subcategory") == "Miscellaneous Investigation",0)\
                               .when(col("subcategory") == "Non - Criminal",0)\
                               .when(col("subcategory") == "Prostitution",0)\
                               .when(col("subcategory") == "Case Closure",0)\
                               .when(col("subcategory") == "Suspicious Occ",0)\
                               .when(col("subcategory") == "Motor Vehicle Theft (Attempted)",1)\
                               .when(col("subcategory") == "Human Trafficking, Involuntary Servitude",6)\
                               .when(col("subcategory") == "Disorderly Conduct",3)\
                               .when(col("subcategory") == "Burglary - Residential",6)\
                               .when(col("subcategory") == "Extortion - Blackmail",5)\
                               .when(col("subcategory") == "Motor Vehicle Theft",5)\
                               .when(col("subcategory") == "Vandalism",2)\
                               .when(col("subcategory") == "Robbery - Commercial",8)\
                               .when(col("subcategory") == "Aggravated Assault",8)\
                               .when(col("subcategory") == "Stolen Property",2)\
                               .otherwise(0))

# 3. Answer to this business question
crimeDF.select("DateAndTime",
                         "Date",
                         "Month",
                         "DayOfMonth",
                         "Time",
                         "Hour",
                         "Year",
                         "DayOfWeek",
                         "ReportDateAndTime",
                         "RowID",
                         "IncidentID",
                         "IncidentNumber",
                         "CADNumber",
                         "ReportTypeCode",
                         "ReportTypeDescription",
                         "Online",
                         "IncidentCode",
                         "Category",
                         "Subcategory",
                         "Description",
                         "Resolution",
                         "Intersection",
                         "IntersectionID",
                         "PoliceDistrict",
                         "Neighborhood",
                         "SupervisorDistrict",
                         "NeighborhoodID",
                         "PoliceDistrictID",
                         "CurrentSupervisorDistricts",
                         "HSOCZones",
                         "OWEDPublicSpaces",
                         "CentralMarket/TenderloinBoundaryPolygon",
                         "ParksAllianceCPSI",
                         "ESNCAG-BoundaryFile",
                         "AreasOfVulnerability",
                         "CrimeSeverity")\
                     .groupBy("CrimeSeverity")\
                     .agg(count("CrimeSeverity").alias("NumOfOccurences"), \
                          (count("CrimeSeverity")/totalCrimes*100).alias("Ratio"))\
                     .orderBy(col("CrimeSeverity").desc())\
                     .select("CrimeSeverity","NumOfOccurences",round("Ratio",2).alias("RoundedRatio")).show()

+------------------------------------------+
|subcategory                               |
+------------------------------------------+
|Burglary - Hot Prowl                      |
|Liquor Law Violation                      |
|Burglary - Commercial                     |
|Vehicle Misplaced                         |
|Child Abuse                               |
|Stalking                                  |
|Larceny Theft - From Building             |
|Drunkenness                               |
|Forgery And Counterfeiting                |
|Robbery - Other                           |
|Sex Offense                               |
|Intimidation                              |
|Fire Report                               |
|Simple Assault                            |
|Larceny Theft - Bicycle                   |
|Larceny - Auto Parts                      |
|Robbery - Carjacking                      |
|Recovered Vehicle                         |
|Drug Violation                            |
|Embezzlem

### B1. Severity of crimes and their location

In [28]:
severeCrimeDF = crimeDF.select("CrimeSeverity", "Intersection")

severeCrimeDF.where((col("CrimeSeverity") == 9) | (col("CrimeSeverity") == 8) | (col("CrimeSeverity") == 7) | (col("CrimeSeverity") == 6))\
.na.drop()\
.groupBy("Intersection")\
.pivot("CrimeSeverity", ("9", "8", "7", "6"))\
.agg(count("CrimeSeverity").alias("NumOfOccurences"))\
.orderBy(col("9").desc(), col("8").desc(), col("7").desc(), col("6").desc())\
.show(20, False)

+-------------------------------------------------+---+----+----+----+
|Intersection                                     |9  |8   |7   |6   |
+-------------------------------------------------+---+----+----+----+
|WILLIAMS AVE \ NEPTUNE ST                        |16 |26  |null|2   |
|SYCAMORE ST \ VALENCIA ST                        |6  |24  |2   |5   |
|04TH ST \ LONG BRIDGE ST                         |5  |25  |2   |13  |
|16TH ST \ MISSION ST                             |4  |108 |1   |2   |
|ELLIS ST \ TAYLOR ST                             |4  |32  |3   |5   |
|FILLMORE ST \ TURK ST                            |4  |19  |null|5   |
|GIRARD ST \ FELTON ST                            |4  |3   |null|null|
|BOARDMAN PL \ BRYANT ST                          |3  |440 |31  |22  |
|17TH ST \ MISSION ST                             |3  |45  |4   |2   |
|ELLIS ST \ LEAVENWORTH ST                        |3  |40  |1   |14  |
|EDDY ST \ HYDE ST                                |3  |40  |1   |7   |
|15TH 

### B2. Severity of crimes and their location including most dangerous days of the week

In [29]:
# It is also interesting to see what day of the week most of these crimes happen in the above areas. 
# Are they all happening around the same days of the week? 

severeCrimeTimedDF = crimeDF.select("CrimeSeverity", "Intersection", "DayOfWeek")

severeCrimeTimedDF.where((col("CrimeSeverity") == 9) | (col("CrimeSeverity") == 8) | (col("CrimeSeverity") == 7) | (col("CrimeSeverity") == 6))\
.na.drop()\
.groupBy("Intersection","DayOfWeek")\
.pivot("CrimeSeverity", ("9", "8", "7", "6"))\
.agg(count("CrimeSeverity").alias("NumOfOccurences"))\
.orderBy(col("9").desc(), col("8").desc(), col("7").desc(), col("6").desc())\
.show(20, False)

+-----------------------------------------+---------+---+----+----+----+
|Intersection                             |DayOfWeek|9  |8   |7   |6   |
+-----------------------------------------+---------+---+----+----+----+
|WILLIAMS AVE \ NEPTUNE ST                |5        |6  |6   |null|null|
|WILLIAMS AVE \ NEPTUNE ST                |6        |4  |2   |null|null|
|WILLIAMS AVE \ NEPTUNE ST                |4        |4  |2   |null|null|
|ELLIS ST \ TAYLOR ST                     |4        |3  |3   |null|2   |
|CALEDONIA ST \ 16TH ST                   |1        |3  |3   |null|null|
|SYCAMORE ST \ VALENCIA ST                |6        |3  |2   |1   |null|
|MIDDLE WEST DR \ MARTIN LUTHER KING JR DR|5        |3  |1   |null|null|
|GOETTINGEN ST \ BURROWS ST               |5        |3  |null|null|null|
|ANGELOS ALY \ JULIA ST \ MISSION ST      |5        |3  |null|null|null|
|16TH ST \ MISSION ST                     |4        |2  |15  |null|1   |
|STEVENSON ST \ 07TH ST \ ODD FELLOWS WAY |7       

### C1. Top 10 dangerous intersections with severe crime

In [30]:
severeCrimeDF.where((col("CrimeSeverity") == 9) | (col("CrimeSeverity") == 8) | (col("CrimeSeverity") == 7) | (col("CrimeSeverity") == 6))\
.na.drop()\
.groupBy("Intersection")\
.agg(count("CrimeSeverity").alias("NumOfOccurences"))\
.orderBy("NumOfOccurences", ascending=False)\
.show(10, False)

+-------------------------------------------------+---------------+
|Intersection                                     |NumOfOccurences|
+-------------------------------------------------+---------------+
|BOARDMAN PL \ BRYANT ST                          |496            |
|MARKET ST \ POWELL ST                            |135            |
|16TH ST \ MISSION ST                             |115            |
|EDDY ST \ CYRIL MAGNIN ST                        |109            |
|POWELL ST \ OFARRELL ST                          |102            |
|ELLIS ST \ POWELL ST                             |90             |
|TAYLOR ST \ 06TH ST \ GOLDEN GATE AVE \ MARKET ST|80             |
|TURK ST \ MARKET ST \ MASON ST                   |79             |
|EDDY ST \ MASON ST                               |76             |
|LEAVENWORTH ST \ TURK ST                         |75             |
+-------------------------------------------------+---------------+
only showing top 10 rows



### C2. Top 10 dangerous intersections with severe crime including most dangerous days of the week

In [31]:
severeCrimeTimedDF.where((col("CrimeSeverity") == 9) | (col("CrimeSeverity") == 8) | (col("CrimeSeverity") == 7) | (col("CrimeSeverity") == 6))\
.na.drop()\
.groupBy("Intersection", "DayOfWeek")\
.agg(count("CrimeSeverity").alias("NumOfOccurences"))\
.orderBy("NumOfOccurences", ascending=False)\
.show(25, False)

+-------------------------------------------------+---------+---------------+
|Intersection                                     |DayOfWeek|NumOfOccurences|
+-------------------------------------------------+---------+---------------+
|BOARDMAN PL \ BRYANT ST                          |2        |117            |
|BOARDMAN PL \ BRYANT ST                          |4        |81             |
|BOARDMAN PL \ BRYANT ST                          |1        |80             |
|BOARDMAN PL \ BRYANT ST                          |3        |69             |
|BOARDMAN PL \ BRYANT ST                          |5        |59             |
|BOARDMAN PL \ BRYANT ST                          |6        |45             |
|BOARDMAN PL \ BRYANT ST                          |7        |45             |
|19TH ST \ MISSION ST                             |6        |28             |
|16TH ST \ MISSION ST                             |7        |25             |
|EDDY ST \ CYRIL MAGNIN ST                        |1        |22 